In [1]:
import torch
from torch import nn

import torchvision
from torchvision.datasets import ImageFolder

from torchvision import transforms

from torch.utils.data import DataLoader
from pathlib import Path
from torchvision.models import vgg16

In [2]:
import sys
sys.path.append("..")

In [3]:
from video_classification.datasets import FolderOfFrameFoldersDataset, FrameWindowDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = Path("/home/ubuntu/SupervisedVideoClassification")
DATA_ROOT = Path(ROOT/"data")

In [6]:
train_transforms = transforms.Compose([
    torchvision.transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

In [7]:
train_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'train', 
                                       transform=train_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)
valid_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'validation', 
                                       transform=valid_transforms, 
                                       base_class=FrameWindowDataset,
                                       window_size=3,
                                       overlapping=True,)

In [8]:
import torch
from torch import nn
from video_classification.models.single_image import SingleImageModel
from video_classification.models.mlp import MLP


class MultiImageModel(nn.Module):
    def __init__(self,
                 window_size=3,
                 single_mlp_sizes=[768, 128],
                 joint_mlp_sizes=[64, 2]):
        super().__init__()
        self.window_size = window_size
        self.single_mlp_sizes = single_mlp_sizes
        self.joint_mlp_sizes = joint_mlp_sizes
        
        self.single_image_model = SingleImageModel(self.single_mlp_sizes)
        self.in_features = self.single_mlp_sizes[-1] * self.window_size
        self.clf = MLP(self.in_features, joint_mlp_sizes)

    def forward(self, x):
        # x is of size [B, T, C, H, W]. In other words, a batch of windows.
        # each img for the same window goes through SingleImageModel
        x = x.transpose(0, 1)  # -> [T, B, C, H, W]
        x = torch.cat([self.single_image_model(window) for window in x], 1)
        # x is now of size [B, T * single_mlp_sizes[-1]]
        
        x = self.clf(x)
        # Now size is [B, joint_mlp_sizes[-1]] which should always be 2

        return x
    
    def freeze_single_image_model(self):
        # Freeze the VGG classifier
        for p in self.single_image_model.parameters():
            p.requires_grad = False
            
    def unfreeze_single_image_model(self):
        # Unfreeze the VGG classifier. Training the whole VGG is a no-go, so we only train the classifier part.
        for p in self.single_image_model.clf.parameters():
            p.requires_grad = True 

In [9]:
model = MultiImageModel(
                 window_size=3,
                 single_mlp_sizes=[1024, 256],
                 joint_mlp_sizes=[128, 2])

model = model.to(device)

In [10]:
x = torch.stack([train_ds[0][0], train_ds[1][0], train_ds[2][0], train_ds[3][0]]).to(device)

In [11]:
model(x)

tensor([[ 0.1265, -0.1215],
        [-0.2513, -0.5642],
        [ 0.0864,  0.3089],
        [-0.2790,  0.4778]], device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
from video_classification.trainer import Trainer

classes_weights = torch.Tensor([0.3, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=classes_weights)

In [13]:
trainer = Trainer(train_ds, 
                  valid_ds, 
                  model, 
                  criterion,
                  "multi_frame_vgg_from_scratch",
                  str(ROOT/'checkpoints'),
                  device=device,
                  amp_opt_level="O1",
                 )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [14]:
# First 3 epochs: only joint MLP unfrozen, high learning rate

trainer.train(lr=1e-3, 
              batch_size=48, 
              n_epochs=20,
              gradient_accumulation_steps=8,
              num_workers=8,
              max_gradient_norm=2.0,
             )

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Training Results - Epoch: 1: Avg accuracy: 0.94 |Precision: 0.96, 0.59 |Recall: 0.97, 0.51 | F1: 0.76 | Avg loss: 0.41
Validation Results - Epoch: 1: Avg accuracy: 0.68 |Precision: 0.93, 0.12 |Recall: 0.70, 0.43 | F1: 0.50 | Avg loss: 0.68


Training Results - Epoch: 2: Avg accuracy: 0.95 |Precision: 0.96, 0.87 |Recall: 0.99, 0.44 | F1: 0.78 | Avg loss: 0.31
Validation Results - Epoch: 2: Avg accuracy: 0.91 |Precision: 0.94, 0.49 |Recall: 0.97, 0.31 | F1: 0.66 | Avg loss: 0.42


Training Results - Epoch: 3: Avg accuracy: 0.96 |Precision: 0.96, 0.96 |Recall: 1.00, 0.43 | F1: 0.79 | Avg loss: 0.29
Validation Results - Epoch: 3: Avg accuracy: 0.87 |Precision: 0.93, 0.27 |Recall: 0.92, 0.31 | F1: 0.61 | Avg loss: 0.50


Training Results - Epoch: 4: Avg accuracy: 0.96 |Precision: 0.96, 0.94 |Recall: 1.00, 0.46 | F1: 0.80 | Avg loss: 0.28
Validation Results - Epoch: 4: Avg accuracy: 0.92 |Precision: 0.93, 0.73 |Recall: 0.99, 0.20 | F1: 0.64 | Avg loss: 0.43


...

Training Results - Epoch: 16: Avg accuracy: 0.96 |Precision: 0.96, 0.96 |Recall: 1.00, 0.45 | F1: 0.80 | Avg loss: 0.25
Validation Results - Epoch: 16: Avg accuracy: 0.92 |Precision: 0.93, 0.64 |Recall: 0.99, 0.24 | F1: 0.65 | Avg loss: 0.42


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Training Results - Epoch: 17: Avg accuracy: 0.96 |Precision: 0.96, 0.95 |Recall: 1.00, 0.51 | F1: 0.82 | Avg loss: 0.24
Validation Results - Epoch: 17: Avg accuracy: 0.92 |Precision: 0.93, 0.59 |Recall: 0.98, 0.29 | F1: 0.67 | Avg loss: 0.44


Training Results - Epoch: 18: Avg accuracy: 0.96 |Precision: 0.96, 0.96 |Recall: 1.00, 0.50 | F1: 0.82 | Avg loss: 0.24
Validation Results - Epoch: 18: Avg accuracy: 0.93 |Precision: 0.93, 0.79 |Recall: 0.99, 0.26 | F1: 0.68 | Avg loss: 0.48


Training Results - Epoch: 19: Avg accuracy: 0.96 |Precision: 0.96, 0.96 |Recall: 1.00, 0.50 | F1: 0.82 | Avg loss: 0.23
Validation Results - Epoch: 19: Avg accuracy: 0.92 |Precision: 0.93, 0.61 |Recall: 0.98, 0.26 | F1: 0.66 | Avg loss: 0.47


KeyboardInterrupt: 

In [16]:
import pandas as pd
reform = {(outerKey, innerKey): values for outerKey, innerDict in trainer.epoch_state.items() for innerKey, values in innerDict.items()}
df = pd.DataFrame(reform).T
df

accuracy        f1       nll  \
1  train  0.937559  0.757145  0.413565   
   test   0.679182  0.496446  0.678223   
2  train  0.954032  0.781115  0.305601   
   test   0.910989  0.664824  0.417051   
3  train  0.956959  0.787474  0.285249   
   test   0.865007  0.607937  0.503948   
4  train   0.95816  0.798594  0.275739   
   test    0.92259  0.635217  0.431444   
5  train   0.95467  0.790076   0.27285   
   test   0.838009  0.579599  0.544394   
6  train  0.954407  0.794968  0.264289   
   test   0.909091  0.649548  0.412052   
7  train  0.954632  0.768462  0.315977   
   test    0.90213   0.61898  0.482983   
8  train  0.957597  0.812269  0.251958   
   test   0.772411  0.542744  0.681547   
9  train  0.952006  0.800895  0.248983   
   test   0.900654  0.637232   0.47173   
10 train  0.956584   0.79405  0.282381   
   test   0.898756  0.606954   0.55602   
11 train   0.95118  0.795584  0.239462   
   test   0.916895  0.663059  0.445089   
12 train   0.95921  0.803399  0.245732   
   test    0.90888  0.639117  0.468172   
13 train  0.960899  0.813636  0.248401   
   test   0.903396  0.650454  0.506134   
14 train  0.960299  0.809197  0.247192   
   test    0.91795  0.661805  0.445093   
15 train  0.963151  0.830082  0.226865   
   test   0.915208   0.67175  0.405801   
16 train  0.958272   0.79645  0.247179   
   test   0.920903  0.651729  0.420588   
17 train  0.961387  0.819008  0.238289   
   test   0.919637  0.673692  0.444475   
18 train  0.961274  0.816938  0.239882   
   test   0.928707  0.679272  0.479077   
19 train  0.961687  0.819174  0.234928   
   test   0.920059  0.660189  0.474437   

                                          precision  \
1  train   [0.9616572414069707, 0.5874125874125874]   
   test   [0.9276335877862596, 0.12414733969986358]   
2  train   [0.9573099415204679, 0.8698698698698699]   
   test    [0.9350736278447122, 0.4942084942084942]   
3  train    [0.956835649406102, 0.9605411499436303]   
   test    [0.9323626115547206, 0.2712215320910973]   
4  train   [0.9588459741473291, 0.9398963730569948]   
   test    [0.9273827534039334, 0.7280701754385965]   
5  train   [0.9592347717225461, 0.8474264705882353]   
   test     [0.929642166344294, 0.2115702479338843]   
6  train   [0.9608466855168081, 0.8160337552742616]   
   test   [0.9324294287619471, 0.47520661157024796]   
7  train   [0.9539056457849961, 0.9784537389100126]   
   test   [0.9280958721704394, 0.40425531914893614]   
8  train   [0.9634803825115108, 0.8368336025848142]   
   test    [0.929081767663403, 0.15696465696465697]   
9  train   [0.9647736298649722, 0.7331518039482642]   
   test    [0.9318232787620543, 0.4078014184397163]   
10 train   [0.9588501443170294, 0.8991097922848664]   
   test   [0.9265097690941385, 0.37130801687763715]   
11 train   [0.9637106369477274, 0.7310104529616724]   
   test    [0.9331715924128805, 0.5603864734299517]   
12 train    [0.959319526627219, 0.9562955254942768]   
   test   [0.9305002213368747, 0.47085201793721976]   
13 train   [0.9608232955209917, 0.9628514056224899]   
   test   [0.9339622641509434, 0.43252595155709345]   
14 train   [0.9600451730986409, 0.9670103092783505]   
   test     [0.9326732673267327, 0.576530612244898]   
15 train   [0.9639096567111598, 0.9455535390199638]   
   test    [0.9353620613060862, 0.5355648535564853]   
16 train     [0.958172983479106, 0.961038961038961]   
   test    [0.9302376280793547, 0.6428571428571429]   
17 train           [0.96203125, 0.9456625357483317]   
   test    [0.9345093715545755, 0.5922330097087378]   
18 train   [0.9614859328052444, 0.9559686888454012]   
   test    [0.9330434782608695, 0.7872340425531915]   
19 train   [0.9617906486613067, 0.9591041869522883]   
   test    [0.9318729463307777, 0.6136363636363636]   

                                             recall  
1  train   [0.9713186145432449, 0.5132382892057027]  
   test    [0.7029153169828783, 0.4343675417661098]  
2  train   [0.9947336439133077, 0.4424643584521385]  
   test  